<a href="https://colab.research.google.com/github/ashishawasthi/colab/blob/master/Llama_Index_Q%26A_with_PDFs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q llama-index langchain transformers

In [ ]:
import torch
from langchain.llms.base import LLM
from llama_index import SimpleDirectoryReader, LangchainEmbedding, GPTListIndex, PromptHelper
from llama_index import LLMPredictor, ServiceContext
from transformers import pipeline
from typing import Optional, List, Mapping, Any

In [ ]:
max_input_size = 2048
num_output = 256
max_chunk_overlap = 20
#chunk_size_limit = 600
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

In [ ]:
class CustomLLM(LLM):
    model_name = "facebook/opt-iml-max-30b"
    pipeline = pipeline("text-generation", model=model_name, device="cuda:0", model_kwargs={"torch_dtype":torch.bfloat16})

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        prompt_length = len(prompt)
        response = self.pipeline(prompt, max_new_tokens=num_output)[0]["generated_text"]

        # only return newly generated tokens
        return response[prompt_length:]

    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"name_of_model": self.model_name}

    @property
    def _llm_type(self) -> str:
        return "custom"

In [ ]:
# define LLM
llm_predictor = LLMPredictor(llm=CustomLLM())

In [ ]:
# Load data
# Mount the file-store, where report files are stored (CIO reports are public information)
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

#/content/gdrive/My Drive/Colab Notebooks/GenAI/reports/
documents = SimpleDirectoryReader('./reports').load_data()

In [ ]:
# Build ListIndex
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)
index = GPTListIndex.from_documents(documents, service_context=service_context)
#index = GPTSimpleVectorIndex.from_documents(documents, service_context=service_context)
#index.save_to_disk('llama-simple-vector-index-reports.json')

In [ ]:
# Query and print response
#index = GPTSimpleVectorIndex.load_from_disk(input_index)
query_engine = index.as_query_engine()
response = query_engine.query("Is Japanese economy doing good?")#, response_mode="compact")
print(response)